# Part 1: Transformers for Sequence Classification

This notebook reproduces the standard Hugging Face token classification workflow using DistilBERT. It serves as a baseline tutorial to understand how transformer-based NER works before applying these concepts to the more complex Hindi Chunking task in Part 2.

The code and data used are the same as in the [token classification tutorial on huggingface](https://huggingface.co/docs/transformers/tasks/token_classification). Minor adjustments were made to manually load the dataset since the original script is deprecated.

## Setup

In [1]:
import numpy as np
import evaluate
from datasets import Dataset, DatasetDict, ClassLabel, Features, Sequence, Value
from transformers import AutoTokenizer, DataCollatorForTokenClassification, AutoModelForTokenClassification, TrainingArguments, Trainer, pipeline
import torch
import requests

## Load WNUT 17 Dataset (Manual Loading)

Since the `wnut_17` dataset script on Hugging Face is no longer supported, the raw files are downloaded and parsed manually.

In [2]:
def load_wnut_data(url):
    response = requests.get(url)
    # processing line by line instead of splitting by double newline
    lines = response.text.split('\n')
    
    tokens_list = []
    ner_tags_list = []
    
    current_tokens = []
    current_tags = []
    
    # Get label list explicitly to map strings to IDs
    _label_list = [
        "O", "B-corporation", "I-corporation", "B-creative-work", "I-creative-work", 
        "B-group", "I-group", "B-location", "I-location", "B-person", "I-person", 
        "B-product", "I-product"
    ]
    label2id = {label: i for i, label in enumerate(_label_list)}
    
    for line in lines:
        # Check if line is empty or just whitespace (some lines are just \t)
        if not line.strip():
            if current_tokens:
                tokens_list.append(current_tokens)
                ner_tags_list.append(current_tags)
                current_tokens = []
                current_tags = []
            continue
            
        parts = line.split('\t')
        if len(parts) >= 2:
            current_tokens.append(parts[0])
            tag = parts[1].strip() # Remove any potential trailing whitespace from tag
            current_tags.append(label2id.get(tag, 0)) # Default to O if unknown
            
    # Add last buffer if exists
    if current_tokens:
        tokens_list.append(current_tokens)
        ner_tags_list.append(current_tags)
            
    return {'tokens': tokens_list, 'ner_tags': ner_tags_list}

urls = {
    'train': 'https://raw.githubusercontent.com/leondz/emerging_entities_17/master/wnut17train.conll',
    'validation': 'https://raw.githubusercontent.com/leondz/emerging_entities_17/master/emerging.dev.conll',
    'test': 'https://raw.githubusercontent.com/leondz/emerging_entities_17/master/emerging.test.annotated'
}

data_splits = {}
for split, url in urls.items():
    print(f"Downloading {split}...")
    data_dict = load_wnut_data(url)
    data_splits[split] = Dataset.from_dict(data_dict)

wnut = DatasetDict(data_splits)
print("Loaded dataset:", wnut)

# Define features manually for compatibility with the rest of the tutorial
label_list = [
    "O",
    "B-corporation",
    "I-corporation",
    "B-creative-work",
    "I-creative-work",
    "B-group",
    "I-group",
    "B-location",
    "I-location",
    "B-person",
    "I-person",
    "B-product",
    "I-product",
]

In [3]:
print(wnut["train"][0])

## Preprocess

In [4]:
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

In [5]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [6]:
tokenized_wnut = wnut.map(tokenize_and_align_labels, batched=True)

In [8]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

## Evaluate

In [9]:
seqeval = evaluate.load("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

## Train

In [10]:
id2label = {i: label for i, label in enumerate(label_list)}
label2id = {label: i for i, label in enumerate(label_list)}

model = AutoModelForTokenClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=13, id2label=id2label, label2id=label2id
)

In [11]:
training_args = TrainingArguments(
    output_dir="my_awesome_wnut_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False, # Disabled as requested
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_wnut["train"],
    eval_dataset=tokenized_wnut["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

## Inference

In [12]:
text = "The Golden State Warriors are an American professional basketball team based in San Francisco."

# Using raw model since we didn't push to hub
inputs = tokenizer(text, return_tensors="pt")
with torch.no_grad():
    logits = model(**inputs.to(model.device)).logits

predictions = torch.argmax(logits, dim=2)
predicted_token_class = [model.config.id2label[t.item()] for t in predictions[0]]
print(predicted_token_class)